In [0]:
# homework
# upload train data 
from google.colab import files
uploaded = files.upload()

Saving train.csv to train.csv


In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import datasets

In [0]:
# load dataset

t_train = pd.read_csv("train.csv")

# Manipulate data

# Drop irrelevant columns
t_train.drop(['Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)

# Drop and fill NA data
t_train.dropna(subset=['Embarked'], inplace = True)
t_train['Age'] = t_train['Age'].fillna(t_train.groupby(['Sex','Pclass'])['Age'].transform('mean'))

# Process the data
t_train['Sex'] = t_train['Sex'].map({'female':0,'male':1}).astype(int)
t_train = t_train.join(pd.get_dummies(t_train.Embarked, prefix = 'Em'))
t_train['Relatives'] = t_train.apply(lambda t_train: t_train['SibSp'] + t_train['Parch'], axis=1)

# Standardize
scaler = preprocessing.StandardScaler()
t_train['st_Pclass'] = scaler.fit_transform(t_train['Pclass'].values.reshape(-1,1))
t_train['st_Age'] = scaler.fit_transform(t_train['Age'].values.reshape(-1,1))
t_train['st_Fare'] = scaler.fit_transform(t_train['Fare'].values.reshape(-1,1))
t_train['st_RL'] = scaler.fit_transform(t_train['Relatives'].values.reshape(-1,1))

t_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 17 columns):
PassengerId    889 non-null int64
Survived       889 non-null int64
Pclass         889 non-null int64
Sex            889 non-null int64
Age            889 non-null float64
SibSp          889 non-null int64
Parch          889 non-null int64
Fare           889 non-null float64
Embarked       889 non-null object
Em_C           889 non-null uint8
Em_Q           889 non-null uint8
Em_S           889 non-null uint8
Relatives      889 non-null int64
st_Pclass      889 non-null float64
st_Age         889 non-null float64
st_Fare        889 non-null float64
st_RL          889 non-null float64
dtypes: float64(6), int64(7), object(1), uint8(3)
memory usage: 146.8+ KB


In [0]:
Y = t_train.Survived
X = t_train.loc[:,['st_Pclass','Sex','st_Age','st_RL','st_Fare','Em_C','Em_Q','Em_S']]
# Generate the model
L_classifier = linear_model.LogisticRegression(class_weight = 'balanced', solver = 'liblinear')
L_classifier.fit(X, Y)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
# upload test data
from google.colab import files
uploaded = files.upload()

In [0]:
t_test = pd.read_csv("test.csv")

# Use the same way to process test data
t_test.drop(['Name', 'Ticket', 'Cabin'], axis = 1, inplace = True)
t_test['Fare'].fillna(t_test['Fare'].mean(), inplace = True)
t_test['Age'].fillna(t_test['Age'].mean(), inplace = True)
t_test['Sex'] = t_test['Sex'].map({'female':0,'male':1}).astype(int)
t_test = t_test.join(pd.get_dummies(t_test.Embarked, prefix = 'Em'))
t_test['Relatives'] = t_test.apply(lambda t_test: t_test['SibSp'] + t_test['Parch'], axis=1)

t_test['st_Pclass'] = scaler.fit_transform(t_test['Pclass'].values.reshape(-1,1))
t_test['st_Age'] = scaler.fit_transform(t_test['Age'].values.reshape(-1,1))
t_test['st_Fare'] = scaler.fit_transform(t_test['Fare'].values.reshape(-1,1))
t_test['st_RL'] = scaler.fit_transform(t_test['Relatives'].values.reshape(-1,1))

t_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null int64
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null object
Em_C           418 non-null uint8
Em_Q           418 non-null uint8
Em_S           418 non-null uint8
Relatives      418 non-null int64
st_Pclass      418 non-null float64
st_Age         418 non-null float64
st_Fare        418 non-null float64
st_RL          418 non-null float64
dtypes: float64(6), int64(6), object(1), uint8(3)
memory usage: 43.8+ KB


In [0]:
X_test = t_test.loc[:,['st_Pclass','Sex','st_Age','st_RL','st_Fare','Em_C','Em_Q','Em_S']]
predictions = L_classifier.predict(X_test)
results = pd.DataFrame({'PassengerId': t_test.PassengerId, 'Survived': predictions})
results.to_csv('Submission.csv', index = False)